# Wavelet Features

## Data Preparation

In [23]:
import os 
import sys

from sklearn.model_selection import train_test_split
import numpy
import pandas
from keras.utils import to_categorical

WORKING_DIR_PATH = globals()['_dh'][0]
WORKING_DIR_PARENT_PATH = os.path.dirname(WORKING_DIR_PATH)
sys.path.insert(1, WORKING_DIR_PARENT_PATH)
from custom_module.utilities import *

In [24]:
tf.__version__

'2.2.0'

In [25]:
import sklearn; sklearn.__version__

'0.21.2'

In [26]:
# load dataset
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features_1.csv', index_col=0)
features

,25th_percentile_db4_cA4,25th_percentile_db4_cD1,25th_percentile_db4_cD2,25th_percentile_db4_cD3,25th_percentile_db4_cD4,25th_percentile_db5_cA4,25th_percentile_db5_cD1,25th_percentile_db5_cD2,25th_percentile_db5_cD3,25th_percentile_db5_cD4,...,zcr_db5_cD3,zcr_db5_cD4,zcr_db8_cA7,zcr_db8_cD1,zcr_db8_cD2,zcr_db8_cD3,zcr_db8_cD4,zcr_db8_cD5,zcr_db8_cD6,zcr_db8_cD7
0,-0.236240,-0.012063,-0.027807,-0.051443,-0.082820,-0.237137,-0.011746,-0.027222,-0.051066,-0.082349,...,48729.0,24515.0,1367.0,236539.0,100477.0,48721.0,24523.0,10601.0,5868.0,2136.0
1,-0.733787,-0.017003,-0.045150,-0.053358,-0.045559,-0.728479,-0.016115,-0.044458,-0.053728,-0.045010,...,40467.0,21751.0,4280.0,232788.0,104560.0,40835.0,21922.0,11688.0,5678.0,4187.0
2,-0.206548,-0.001990,-0.004977,-0.010344,-0.032882,-0.206663,-0.001933,-0.004869,-0.009837,-0.031671,...,51110.0,28460.0,3429.0,238953.0,98627.0,50426.0,28197.0,12380.0,5750.0,2853.0
3,-0.505051,-0.017088,-0.050502,-0.085439,-0.155965,-0.503971,-0.016430,-0.050478,-0.080363,-0.153229,...,46009.0,24784.0,4084.0,239878.0,106315.0,46383.0,25001.0,10550.0,6405.0,3401.0
4,-0.265025,-0.007068,-0.017015,-0.033292,-0.048008,-0.263057,-0.006823,-0.016241,-0.032975,-0.046946,...,50126.0,21473.0,3355.0,235508.0,96446.0,50054.0,21542.0,11100.0,6028.0,3176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,-0.104572,-0.000267,-0.001287,-0.004655,-0.018606,-0.104197,-0.000241,-0.001230,-0.004309,-0.018012,...,111718.0,57071.0,5266.0,478728.0,194404.0,110085.0,56729.0,26120.0,10705.0,4714.0
3333,-0.064313,-0.000214,-0.000820,-0.004044,-0.013585,-0.064493,-0.000202,-0.000685,-0.003693,-0.013236,...,100899.0,57721.0,5961.0,378640.0,180357.0,98690.0,57296.0,25947.0,13375.0,3125.0
3334,-0.477425,-0.000967,-0.004003,-0.008481,-0.038837,-0.476932,-0.000873,-0.004009,-0.007479,-0.036333,...,95367.0,62597.0,3067.0,491052.0,210852.0,92628.0,61759.0,24119.0,12008.0,6321.0
3335,-0.328624,-0.001875,-0.008336,-0.021885,-0.049970,-0.329515,-0.001693,-0.008131,-0.021180,-0.048207,...,101933.0,50380.0,5290.0,483699.0,219884.0,100631.0,49950.0,30214.0,9810.0,5397.0


In [27]:
# shuffle dataset
features = features.reindex(numpy.random.permutation(features.index))
features

,25th_percentile_db4_cA4,25th_percentile_db4_cD1,25th_percentile_db4_cD2,25th_percentile_db4_cD3,25th_percentile_db4_cD4,25th_percentile_db5_cA4,25th_percentile_db5_cD1,25th_percentile_db5_cD2,25th_percentile_db5_cD3,25th_percentile_db5_cD4,...,zcr_db5_cD3,zcr_db5_cD4,zcr_db8_cA7,zcr_db8_cD1,zcr_db8_cD2,zcr_db8_cD3,zcr_db8_cD4,zcr_db8_cD5,zcr_db8_cD6,zcr_db8_cD7
3049,-0.785576,-0.008296,-0.039609,-0.101652,-0.181327,-0.782742,-0.007366,-0.038585,-0.102132,-0.181910,...,101406.0,47880.0,4837.0,456953.0,235647.0,101777.0,47765.0,26760.0,10303.0,6670.0
2242,-0.726786,-0.000231,-0.002101,-0.020969,-0.198744,-0.726736,-0.000180,-0.001577,-0.014590,-0.188684,...,92414.0,68127.0,7787.0,396331.0,177409.0,85459.0,67993.0,14318.0,13220.0,6937.0
2887,-1.673391,-0.002072,-0.015191,-0.073074,-0.240770,-1.676405,-0.001729,-0.014246,-0.070403,-0.240090,...,116357.0,55497.0,7043.0,434298.0,227846.0,114973.0,55347.0,24673.0,13972.0,5703.0
1959,-0.314677,-0.001478,-0.008460,-0.024828,-0.052989,-0.313523,-0.001275,-0.008145,-0.024960,-0.052882,...,106888.0,48171.0,6005.0,452388.0,241160.0,106719.0,47945.0,24737.0,13255.0,6652.0
307,-0.106518,-0.004947,-0.012770,-0.018741,-0.031234,-0.106345,-0.004793,-0.012666,-0.018499,-0.029976,...,43013.0,25968.0,2437.0,246066.0,107176.0,43220.0,26158.0,12823.0,6140.0,2648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,-0.584666,-0.006704,-0.023665,-0.058747,-0.102654,-0.582680,-0.006324,-0.023077,-0.058495,-0.100136,...,102648.0,47341.0,6309.0,491905.0,217728.0,101871.0,47306.0,24867.0,13529.0,6190.0
3192,-0.443331,-0.003047,-0.018994,-0.061773,-0.139975,-0.441565,-0.002547,-0.017681,-0.060793,-0.136950,...,103857.0,53248.0,6627.0,430745.0,233642.0,103506.0,53448.0,23343.0,11347.0,5866.0
1704,-0.633330,-0.001672,-0.013848,-0.068424,-0.182894,-0.633093,-0.001321,-0.012269,-0.067776,-0.179862,...,119558.0,50533.0,5582.0,406852.0,224912.0,118990.0,50307.0,26633.0,10101.0,6299.0
3150,-0.506734,-0.003485,-0.012475,-0.039650,-0.096496,-0.506390,-0.003306,-0.011996,-0.038122,-0.093512,...,109217.0,49689.0,7456.0,471633.0,214794.0,107929.0,49403.0,28259.0,10545.0,5216.0


In [28]:
# split dataset into target (y) and predictors (x)
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]

In [29]:
# get wavelet predictors
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values
wavelet_predictors

,25th_percentile_db4_cA4,25th_percentile_db4_cD1,25th_percentile_db4_cD2,25th_percentile_db4_cD3,25th_percentile_db4_cD4,25th_percentile_db5_cA4,25th_percentile_db5_cD1,25th_percentile_db5_cD2,25th_percentile_db5_cD3,25th_percentile_db5_cD4,...,zcr_db5_cD3,zcr_db5_cD4,zcr_db8_cA7,zcr_db8_cD1,zcr_db8_cD2,zcr_db8_cD3,zcr_db8_cD4,zcr_db8_cD5,zcr_db8_cD6,zcr_db8_cD7
3049,-0.785576,-0.008296,-0.039609,-0.101652,-0.181327,-0.782742,-0.007366,-0.038585,-0.102132,-0.181910,...,101406.0,47880.0,4837.0,456953.0,235647.0,101777.0,47765.0,26760.0,10303.0,6670.0
2242,-0.726786,-0.000231,-0.002101,-0.020969,-0.198744,-0.726736,-0.000180,-0.001577,-0.014590,-0.188684,...,92414.0,68127.0,7787.0,396331.0,177409.0,85459.0,67993.0,14318.0,13220.0,6937.0
2887,-1.673391,-0.002072,-0.015191,-0.073074,-0.240770,-1.676405,-0.001729,-0.014246,-0.070403,-0.240090,...,116357.0,55497.0,7043.0,434298.0,227846.0,114973.0,55347.0,24673.0,13972.0,5703.0
1959,-0.314677,-0.001478,-0.008460,-0.024828,-0.052989,-0.313523,-0.001275,-0.008145,-0.024960,-0.052882,...,106888.0,48171.0,6005.0,452388.0,241160.0,106719.0,47945.0,24737.0,13255.0,6652.0
307,-0.106518,-0.004947,-0.012770,-0.018741,-0.031234,-0.106345,-0.004793,-0.012666,-0.018499,-0.029976,...,43013.0,25968.0,2437.0,246066.0,107176.0,43220.0,26158.0,12823.0,6140.0,2648.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3320,-0.584666,-0.006704,-0.023665,-0.058747,-0.102654,-0.582680,-0.006324,-0.023077,-0.058495,-0.100136,...,102648.0,47341.0,6309.0,491905.0,217728.0,101871.0,47306.0,24867.0,13529.0,6190.0
3192,-0.443331,-0.003047,-0.018994,-0.061773,-0.139975,-0.441565,-0.002547,-0.017681,-0.060793,-0.136950,...,103857.0,53248.0,6627.0,430745.0,233642.0,103506.0,53448.0,23343.0,11347.0,5866.0
1704,-0.633330,-0.001672,-0.013848,-0.068424,-0.182894,-0.633093,-0.001321,-0.012269,-0.067776,-0.179862,...,119558.0,50533.0,5582.0,406852.0,224912.0,118990.0,50307.0,26633.0,10101.0,6299.0
3150,-0.506734,-0.003485,-0.012475,-0.039650,-0.096496,-0.506390,-0.003306,-0.011996,-0.038122,-0.093512,...,109217.0,49689.0,7456.0,471633.0,214794.0,107929.0,49403.0,28259.0,10545.0,5216.0


In [30]:
# get wavelet predictor labels with and without outliers
wavelet_predictor_labels_wf_outliers= numpy.asarray([
    '25th_percentile_db8_cA7', '5th_percentile_db8_cA7', '5th_percentile_db8_cD7',
    '75th_percentile_db8_cA7', 'mcr_db4_cD1', 'mcr_db4_cD2', 'mcr_db8_cD2',
    'mcr_db8_cD1', 'mcr_db5_cD2','mcr_db5_cD1','zcr_db8_cD1'])
wavelet_predictor_labels_wo_outliers = [i for i in wavelet_predictor_labels \
                                 if i not in wavelet_predictor_labels_wf_outliers]

In [31]:
# Create standardization and normalization pipelines
preprocess_sp = standardization_pipeline(
    wavelet_predictor_labels,
    wavelet_predictor_labels_wf_outliers,
    wavelet_predictor_labels_wo_outliers)

preprocess_np = normalization_pipeline(
    wavelet_predictor_labels)

In [32]:
# split into sets: training 80% & testing 20% of total dataset
X_train, X_test, y_train, y_test = train_test_split(
    wavelet_predictors, y, test_size=0.2, shuffle=True)

In [35]:
_, ncols = X_train.shape
ncols

252

In [36]:
# one hot encoding our labels
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [37]:
y_train.shape, y_test.shape

((2669, 3), (668, 3))

## Predictive Modelling

In [16]:
param_dist = {
    'clf__n_hidden': [1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__lr': [3, 1, .3, .1, .03, .01, .003, .001, .0003, .0001],
    'clf__units': numpy.arange(4,100)
}

### FNN With Standardized Input

In [17]:
(fnn1_best_score, fnn1_best_params, fnn1_best_estimator) = \
train_model(X_train, y_train, 'fnn_1', ncols, set_shape_create_model,
            pipeline_estimator_sp, param_dist)

Model: "fnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 252)               0         
_________________________________________________________________
dense (Dense)                (None, 30)                7590      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 93        
Total params: 8,613
Trainable params: 8,613
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
45/45 [==============================] - 0s 6ms/step - loss: 103.9264 - accuracy: 0.3647 - val_loss: 1.6662 - val_accuracy: 0.3511
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 1.4061 - accuracy: 0.3457 - val_loss

45/45 [==============================] - 0s 8ms/step - loss: 1.1767 - accuracy: 0.3287 - val_loss: 1.1846 - val_accuracy: 0.3202
Epoch 13/100
45/45 [==============================] - 0s 9ms/step - loss: 1.1428 - accuracy: 0.3371 - val_loss: 1.1148 - val_accuracy: 0.3202
Epoch 14/100
45/45 [==============================] - 0s 6ms/step - loss: 1.1400 - accuracy: 0.3448 - val_loss: 1.1494 - val_accuracy: 0.3933
Epoch 15/100
45/45 [==============================] - 0s 7ms/step - loss: 1.1402 - accuracy: 0.3504 - val_loss: 1.2241 - val_accuracy: 0.3933
Epoch 16/100
45/45 [==============================] - 0s 8ms/step - loss: 1.1417 - accuracy: 0.3160 - val_loss: 1.2092 - val_accuracy: 0.3933
Epoch 17/100
45/45 [==============================] - 0s 7ms/step - loss: 1.1317 - accuracy: 0.3638 - val_loss: 1.1852 - val_accuracy: 0.3933
Epoch 18/100
45/45 [==============================] - 0s 5ms/step - loss: 1.1244 - accuracy: 0.3399 - val_loss: 1.2485 - val_accuracy: 0.3202
Epoch 19/100
45/45 

45/45 [==============================] - 0s 3ms/step - loss: 215.3897 - accuracy: 0.3345 - val_loss: 12.4820 - val_accuracy: 0.3596
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 6.6621 - accuracy: 0.3197 - val_loss: 2.0449 - val_accuracy: 0.3596
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 1.2804 - accuracy: 0.3268 - val_loss: 1.0990 - val_accuracy: 0.3315
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 1.1058 - accuracy: 0.3303 - val_loss: 1.1080 - val_accuracy: 0.3596
Epoch 7/100
45/45 [==============================] - 0s 3ms/step - loss: 1.1013 - accuracy: 0.3380 - val_loss: 1.1127 - val_accuracy: 0.3090
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 1.1025 - accuracy: 0.3394 - val_loss: 1.0995 - val_accuracy: 0.3090
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 1.1054 - accuracy: 0.3345 - val_loss: 1.1013 - val_accuracy: 0.3315
Epoch 10/100
45/45 [==

Epoch 8/100
45/45 [==============================] - 0s 2ms/step - loss: 23.0741 - accuracy: 0.3865 - val_loss: 53.7138 - val_accuracy: 0.3567
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 42.8722 - accuracy: 0.4350 - val_loss: 15.4648 - val_accuracy: 0.4101
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 26.7501 - accuracy: 0.4385 - val_loss: 46.4459 - val_accuracy: 0.3961
Epoch 11/100
45/45 [==============================] - 0s 2ms/step - loss: 17.0325 - accuracy: 0.4800 - val_loss: 33.8168 - val_accuracy: 0.4017
Epoch 12/100
28/28 [==============================] - 0s 1ms/step - loss: 8.8355 - accuracy: 0.4640
Model: "fnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_11 (Flatten)         (None, 252)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 31)        

Epoch 4/100
45/45 [==============================] - 0s 4ms/step - loss: 1.3824 - accuracy: 0.6240 - val_loss: 2.5424 - val_accuracy: 0.5225
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 1.2328 - accuracy: 0.6451 - val_loss: 3.0326 - val_accuracy: 0.5562
Epoch 6/100
45/45 [==============================] - 0s 4ms/step - loss: 1.1939 - accuracy: 0.6360 - val_loss: 2.5022 - val_accuracy: 0.5197
Epoch 7/100
45/45 [==============================] - 0s 6ms/step - loss: 0.8829 - accuracy: 0.6655 - val_loss: 1.8829 - val_accuracy: 0.5871
Epoch 8/100
45/45 [==============================] - 0s 4ms/step - loss: 0.8285 - accuracy: 0.6936 - val_loss: 2.2518 - val_accuracy: 0.5393
Epoch 9/100
45/45 [==============================] - 0s 6ms/step - loss: 0.7496 - accuracy: 0.7091 - val_loss: 2.4108 - val_accuracy: 0.6208
Epoch 10/100
45/45 [==============================] - 0s 7ms/step - loss: 0.8575 - accuracy: 0.7182 - val_loss: 1.9096 - val_accuracy: 0.5702
Epoch 11/100

dense_53 (Dense)             (None, 9)                 90        
_________________________________________________________________
dense_54 (Dense)             (None, 3)                 30        
Total params: 2,487
Trainable params: 2,487
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
45/45 [==============================] - 0s 5ms/step - loss: 0.9688 - accuracy: 0.5432 - val_loss: 0.9288 - val_accuracy: 0.5449
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8206 - accuracy: 0.6219 - val_loss: 0.9031 - val_accuracy: 0.5674
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 0.7815 - accuracy: 0.6381 - val_loss: 0.8485 - val_accuracy: 0.6376
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.7455 - accuracy: 0.6543 - val_loss: 0.8366 - val_accuracy: 0.6067
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.7224 - accuracy: 0.6753 

45/45 [==============================] - 0s 2ms/step - loss: 0.5862 - accuracy: 0.7554 - val_loss: 0.8226 - val_accuracy: 0.6489
Epoch 27/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5790 - accuracy: 0.7569 - val_loss: 0.8123 - val_accuracy: 0.6685
Epoch 28/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5778 - accuracy: 0.7533 - val_loss: 0.8419 - val_accuracy: 0.6685
Epoch 29/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5739 - accuracy: 0.7505 - val_loss: 0.8484 - val_accuracy: 0.6601
Epoch 30/100
45/45 [==============================] - 0s 2ms/step - loss: 0.5702 - accuracy: 0.7688 - val_loss: 0.8952 - val_accuracy: 0.6601
Epoch 31/100
28/28 [==============================] - 0s 1ms/step - loss: 0.8702 - accuracy: 0.6528
Model: "fnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_19 (Flatten)         (None, 252)               0

Epoch 3/100
45/45 [==============================] - 0s 5ms/step - loss: 20.8957 - accuracy: 0.5576 - val_loss: 11.1067 - val_accuracy: 0.5646
Epoch 4/100
45/45 [==============================] - 0s 4ms/step - loss: 14.2920 - accuracy: 0.5969 - val_loss: 14.9753 - val_accuracy: 0.4522
Epoch 5/100
45/45 [==============================] - 0s 5ms/step - loss: 8.1064 - accuracy: 0.5576 - val_loss: 7.0251 - val_accuracy: 0.5674
Epoch 6/100
45/45 [==============================] - 0s 4ms/step - loss: 3.4007 - accuracy: 0.6243 - val_loss: 6.8334 - val_accuracy: 0.4494
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 5.1705 - accuracy: 0.6039 - val_loss: 2.9579 - val_accuracy: 0.5590
Epoch 8/100
45/45 [==============================] - 0s 7ms/step - loss: 1.5703 - accuracy: 0.6138 - val_loss: 2.4884 - val_accuracy: 0.5646
Epoch 9/100
45/45 [==============================] - 0s 5ms/step - loss: 2.0723 - accuracy: 0.6243 - val_loss: 9.9372 - val_accuracy: 0.5337
Epoch 10/

45/45 [==============================] - 0s 6ms/step - loss: 0.5274 - accuracy: 0.7822 - val_loss: 0.9271 - val_accuracy: 0.5955
Epoch 11/100
45/45 [==============================] - 0s 5ms/step - loss: 0.5167 - accuracy: 0.7850 - val_loss: 0.9647 - val_accuracy: 0.6376
Epoch 12/100
45/45 [==============================] - 0s 6ms/step - loss: 0.4933 - accuracy: 0.7906 - val_loss: 0.9470 - val_accuracy: 0.5871
Epoch 13/100
45/45 [==============================] - 0s 5ms/step - loss: 0.4698 - accuracy: 0.8159 - val_loss: 0.9148 - val_accuracy: 0.6348
Epoch 14/100
45/45 [==============================] - 0s 4ms/step - loss: 0.4597 - accuracy: 0.8060 - val_loss: 0.9366 - val_accuracy: 0.6067
Epoch 15/100
28/28 [==============================] - 0s 2ms/step - loss: 0.8388 - accuracy: 0.6539
Model: "fnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_28 (Flatten)         (None, 252)               0

### FNN With Normalized Input

In [18]:
(fnn2_best_score, fnn2_best_params, fnn2_best_estimator) = \
train_model(X_train, y_train, 'fnn_2', ncols, set_shape_create_model,
            pipeline_estimator_np, param_dist)

Model: "fnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 252)               0         
_________________________________________________________________
dense_109 (Dense)            (None, 47)                11891     
_________________________________________________________________
dense_110 (Dense)            (None, 47)                2256      
_________________________________________________________________
dense_111 (Dense)            (None, 47)                2256      
_________________________________________________________________
dense_112 (Dense)            (None, 3)                 144       
Total params: 16,547
Trainable params: 16,547
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
45/45 [==============================] - 1s 27ms/step - loss: 1.0780 - accuracy: 0.4259 - val_loss: 1.0741

Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8881 - accuracy: 0.5692 - val_loss: 0.9845 - val_accuracy: 0.5112
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8852 - accuracy: 0.5840 - val_loss: 0.9015 - val_accuracy: 0.6067
Epoch 7/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8720 - accuracy: 0.5868 - val_loss: 0.8884 - val_accuracy: 0.5983
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8587 - accuracy: 0.5973 - val_loss: 0.8754 - val_accuracy: 0.5899
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8514 - accuracy: 0.6086 - val_loss: 0.8810 - val_accuracy: 0.5702
Epoch 10/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8522 - accuracy: 0.6022 - val_loss: 0.8590 - val_accuracy: 0.5955
Epoch 11/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8328 - accuracy: 0.6177 - val_loss: 0.8388 - val_accuracy: 0.6152
Epoch 12/10

Epoch 11/100
45/45 [==============================] - 0s 5ms/step - loss: 0.8363 - accuracy: 0.6124 - val_loss: 0.8030 - val_accuracy: 0.6545
Epoch 12/100
45/45 [==============================] - 0s 4ms/step - loss: 0.8064 - accuracy: 0.6173 - val_loss: 0.8152 - val_accuracy: 0.6236
Epoch 13/100
45/45 [==============================] - 0s 4ms/step - loss: 0.8064 - accuracy: 0.6166 - val_loss: 0.7820 - val_accuracy: 0.6601
Epoch 14/100
45/45 [==============================] - 0s 7ms/step - loss: 0.7931 - accuracy: 0.6313 - val_loss: 0.9229 - val_accuracy: 0.5618
Epoch 15/100
45/45 [==============================] - 0s 6ms/step - loss: 0.7980 - accuracy: 0.6355 - val_loss: 0.7705 - val_accuracy: 0.6798
Epoch 16/100
45/45 [==============================] - 0s 4ms/step - loss: 0.7920 - accuracy: 0.6362 - val_loss: 0.7786 - val_accuracy: 0.6657
Epoch 17/100
45/45 [==============================] - 0s 4ms/step - loss: 0.7880 - accuracy: 0.6390 - val_loss: 0.8237 - val_accuracy: 0.6348
Epoch 

Epoch 12/100
45/45 [==============================] - 1s 26ms/step - loss: 3.6360 - accuracy: 0.3450 - val_loss: 3.0837 - val_accuracy: 0.3596
Epoch 13/100
45/45 [==============================] - 1s 18ms/step - loss: 3.7297 - accuracy: 0.3387 - val_loss: 1.6210 - val_accuracy: 0.3315
Epoch 14/100
45/45 [==============================] - 1s 19ms/step - loss: 3.8751 - accuracy: 0.3289 - val_loss: 5.8096 - val_accuracy: 0.3596
Epoch 15/100
45/45 [==============================] - 0s 6ms/step - loss: 3.7926 - accuracy: 0.3331 - val_loss: 4.0038 - val_accuracy: 0.3090
Epoch 16/100
45/45 [==============================] - 1s 14ms/step - loss: 3.7149 - accuracy: 0.3387 - val_loss: 5.4128 - val_accuracy: 0.3596
Epoch 17/100
45/45 [==============================] - 0s 10ms/step - loss: 3.7404 - accuracy: 0.3317 - val_loss: 2.9983 - val_accuracy: 0.3596
Epoch 18/100
45/45 [==============================] - 1s 12ms/step - loss: 3.6957 - accuracy: 0.3282 - val_loss: 6.2726 - val_accuracy: 0.3090


45/45 [==============================] - 0s 4ms/step - loss: 1.0905 - accuracy: 0.3521 - val_loss: 1.0940 - val_accuracy: 0.3118
Epoch 51/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0904 - accuracy: 0.3521 - val_loss: 1.0939 - val_accuracy: 0.3118
Epoch 52/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0903 - accuracy: 0.3521 - val_loss: 1.0938 - val_accuracy: 0.3118
Epoch 53/100
45/45 [==============================] - 0s 4ms/step - loss: 1.0901 - accuracy: 0.3521 - val_loss: 1.0937 - val_accuracy: 0.3118
Epoch 54/100
45/45 [==============================] - 0s 4ms/step - loss: 1.0900 - accuracy: 0.3521 - val_loss: 1.0936 - val_accuracy: 0.3118
Epoch 55/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0899 - accuracy: 0.3521 - val_loss: 1.0935 - val_accuracy: 0.3118
Epoch 56/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0898 - accuracy: 0.3521 - val_loss: 1.0934 - val_accuracy: 0.3118
Epoch 57/100
45/45 

...

Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 6.4277 - accuracy: 0.3422 - val_loss: 5.9360 - val_accuracy: 0.3596
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 7.3602 - accuracy: 0.3507 - val_loss: 10.3433 - val_accuracy: 0.3315
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 6.3954 - accuracy: 0.3521 - val_loss: 4.7994 - val_accuracy: 0.3090
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 6.7647 - accuracy: 0.3261 - val_loss: 7.3074 - val_accuracy: 0.3090
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 6.7793 - accuracy: 0.3359 - val_loss: 5.2852 - val_accuracy: 0.3315
Epoch 10/100
45/45 [==============================] - 0s 3ms/step - loss: 6.7959 - accuracy: 0.3268 - val_loss: 2.9879 - val_accuracy: 0.3596
Epoch 11/100
45/45 [==============================] - 0s 4ms/step - loss: 6.4425 - accuracy: 0.3113 - val_loss: 10.5959 - val_accuracy: 0.3315
Epoch 12/

45/45 [==============================] - 0s 3ms/step - loss: 1.1434 - accuracy: 0.3617 - val_loss: 1.1948 - val_accuracy: 0.4916
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0656 - accuracy: 0.4796 - val_loss: 1.3288 - val_accuracy: 0.3652
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0723 - accuracy: 0.4572 - val_loss: 0.9539 - val_accuracy: 0.4888
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 0.9359 - accuracy: 0.5492 - val_loss: 1.2490 - val_accuracy: 0.3876
Epoch 7/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0539 - accuracy: 0.4691 - val_loss: 0.9452 - val_accuracy: 0.5197
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0494 - accuracy: 0.4853 - val_loss: 0.8975 - val_accuracy: 0.5983
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0400 - accuracy: 0.5070 - val_loss: 0.8978 - val_accuracy: 0.5787
Epoch 10/100
45/45 [=====

Epoch 20/100
28/28 [==============================] - 0s 1ms/step - loss: 1.2551 - accuracy: 0.3236
Model: "fnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_59 (Flatten)         (None, 252)               0         
_________________________________________________________________
dense_207 (Dense)            (None, 95)                24035     
_________________________________________________________________
dense_208 (Dense)            (None, 3)                 288       
Total params: 24,323
Trainable params: 24,323
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
45/45 [==============================] - 0s 4ms/step - loss: 14143.6035 - accuracy: 0.3268 - val_loss: 1.9590 - val_accuracy: 0.3624
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 2.1601 - accuracy: 0.3204 - val_loss: 2.4700 - val_accuracy: 0.3

Epoch 5/100
67/67 [==============================] - 0s 6ms/step - loss: 0.8670 - accuracy: 0.5944 - val_loss: 0.8432 - val_accuracy: 0.6180
Epoch 6/100
67/67 [==============================] - 0s 6ms/step - loss: 0.8437 - accuracy: 0.5977 - val_loss: 0.8545 - val_accuracy: 0.5936
Epoch 7/100
67/67 [==============================] - 0s 6ms/step - loss: 0.8429 - accuracy: 0.6061 - val_loss: 0.8274 - val_accuracy: 0.6086
Epoch 8/100
67/67 [==============================] - 0s 6ms/step - loss: 0.8373 - accuracy: 0.6066 - val_loss: 0.8487 - val_accuracy: 0.6161
Epoch 9/100
67/67 [==============================] - 0s 6ms/step - loss: 0.8299 - accuracy: 0.6019 - val_loss: 0.8549 - val_accuracy: 0.5955
Epoch 10/100
67/67 [==============================] - 0s 5ms/step - loss: 0.8142 - accuracy: 0.6155 - val_loss: 0.8395 - val_accuracy: 0.6105
Epoch 11/100
67/67 [==============================] - 0s 7ms/step - loss: 0.8165 - accuracy: 0.6398 - val_loss: 0.7884 - val_accuracy: 0.6479
Epoch 12/10

Epoch 63/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7133 - accuracy: 0.6689 - val_loss: 0.8417 - val_accuracy: 0.6217
Epoch 64/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7214 - accuracy: 0.6745 - val_loss: 0.7865 - val_accuracy: 0.6742
Epoch 65/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7140 - accuracy: 0.6801 - val_loss: 0.7459 - val_accuracy: 0.6779
Epoch 66/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7166 - accuracy: 0.6796 - val_loss: 0.8046 - val_accuracy: 0.6348
Epoch 67/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7084 - accuracy: 0.6876 - val_loss: 0.7390 - val_accuracy: 0.6742
Epoch 68/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7118 - accuracy: 0.6848 - val_loss: 0.7362 - val_accuracy: 0.6742
Epoch 69/100
67/67 [==============================] - 0s 2ms/step - loss: 0.7230 - accuracy: 0.6735 - val_loss: 0.7712 - val_accuracy: 0.6854
Epoch 

### CNN with Standardized Input

In [38]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__units': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25],
    'clf__lr': [3, 1, .3, .1, .03, .01, .003, .001, .0003, .0001],
}

In [39]:
(cnn1_best_score, cnn1_best_params, cnn1_best_estimator) = \
train_model(X_train, y_train, 'cnn_1', ncols, set_shape_create_cnn_model,
            pipeline_estimator_sp, param_dist)

Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 250, 64)           256       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 125, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 123, 128)          24704     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 61, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 59, 192)           73920     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 29, 192)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 27, 256)           147712

Epoch 6/100
45/45 [==============================] - 8s 175ms/step - loss: 3.2032 - accuracy: 0.3204 - val_loss: 1.1093 - val_accuracy: 0.3062
Epoch 7/100
45/45 [==============================] - 9s 191ms/step - loss: 3.0446 - accuracy: 0.3310 - val_loss: 1.1691 - val_accuracy: 0.3708
Epoch 8/100
45/45 [==============================] - 9s 191ms/step - loss: 2.9739 - accuracy: 0.3450 - val_loss: 1.2721 - val_accuracy: 0.3708
Epoch 9/100
45/45 [==============================] - 8s 174ms/step - loss: 2.6627 - accuracy: 0.3289 - val_loss: 1.2160 - val_accuracy: 0.3062
Epoch 10/100
45/45 [==============================] - 8s 176ms/step - loss: 2.8361 - accuracy: 0.3514 - val_loss: 1.1918 - val_accuracy: 0.3230
Epoch 11/100
45/45 [==============================] - 8s 176ms/step - loss: 18663472.0000 - accuracy: 0.3310 - val_loss: 1.4954 - val_accuracy: 0.3708
Epoch 12/100
45/45 [==============================] - 9s 205ms/step - loss: 2.2868 - accuracy: 0.3408 - val_loss: 1.1003 - val_accura

Epoch 10/100
45/45 [==============================] - 3s 68ms/step - loss: 0.4256 - accuracy: 0.8201 - val_loss: 1.0294 - val_accuracy: 0.6180
Epoch 11/100
45/45 [==============================] - 3s 75ms/step - loss: 0.3509 - accuracy: 0.8538 - val_loss: 1.1719 - val_accuracy: 0.6124
Epoch 12/100
45/45 [==============================] - 3s 70ms/step - loss: 0.3672 - accuracy: 0.8398 - val_loss: 1.3648 - val_accuracy: 0.6124
Epoch 13/100
28/28 [==============================] - 1s 23ms/step - loss: 0.8595 - accuracy: 0.6562
Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 250, 64)           256       
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 125, 64)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 123, 128)          2470

Epoch 1/100
45/45 [==============================] - 3s 56ms/step - loss: 0.9674 - accuracy: 0.5320 - val_loss: 0.8718 - val_accuracy: 0.6011
Epoch 2/100
45/45 [==============================] - 2s 39ms/step - loss: 0.7964 - accuracy: 0.6592 - val_loss: 0.7970 - val_accuracy: 0.6236
Epoch 3/100
45/45 [==============================] - 1s 31ms/step - loss: 0.7473 - accuracy: 0.6746 - val_loss: 0.7751 - val_accuracy: 0.6180
Epoch 4/100
45/45 [==============================] - 2s 34ms/step - loss: 0.7067 - accuracy: 0.7013 - val_loss: 0.7963 - val_accuracy: 0.6404
Epoch 5/100
45/45 [==============================] - 2s 37ms/step - loss: 0.6815 - accuracy: 0.7006 - val_loss: 0.7965 - val_accuracy: 0.6433
Epoch 6/100
45/45 [==============================] - 2s 36ms/step - loss: 0.6302 - accuracy: 0.7393 - val_loss: 0.8207 - val_accuracy: 0.6320
Epoch 7/100
45/45 [==============================] - 1s 31ms/step - loss: 0.6227 - accuracy: 0.7379 - val_loss: 0.7993 - val_accuracy: 0.6208
Epoch 

45/45 [==============================] - 5s 110ms/step - loss: 0.5750 - accuracy: 0.7526 - val_loss: 0.8571 - val_accuracy: 0.6096
Epoch 9/100
45/45 [==============================] - 5s 105ms/step - loss: 0.5554 - accuracy: 0.7569 - val_loss: 0.8743 - val_accuracy: 0.6236
Epoch 10/100
45/45 [==============================] - 4s 80ms/step - loss: 0.5710 - accuracy: 0.7667 - val_loss: 0.9294 - val_accuracy: 0.6067
Epoch 11/100
45/45 [==============================] - 3s 76ms/step - loss: 0.5399 - accuracy: 0.7695 - val_loss: 1.0338 - val_accuracy: 0.6404
Epoch 12/100
45/45 [==============================] - 5s 107ms/step - loss: 0.4677 - accuracy: 0.8124 - val_loss: 1.0883 - val_accuracy: 0.6039
Epoch 13/100
45/45 [==============================] - 5s 118ms/step - loss: 0.4049 - accuracy: 0.8292 - val_loss: 1.0399 - val_accuracy: 0.6264
Epoch 14/100
45/45 [==============================] - 6s 128ms/step - loss: 0.3740 - accuracy: 0.8292 - val_loss: 1.0127 - val_accuracy: 0.6208
Epoch 15

Epoch 14/100
45/45 [==============================] - 3s 75ms/step - loss: 0.3842 - accuracy: 0.8525 - val_loss: 1.0612 - val_accuracy: 0.6320
Epoch 15/100
45/45 [==============================] - 3s 68ms/step - loss: 0.3358 - accuracy: 0.8666 - val_loss: 1.2424 - val_accuracy: 0.5871
Epoch 16/100
28/28 [==============================] - 1s 19ms/step - loss: 0.7933 - accuracy: 0.6434
Model: "cnn_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_84 (Conv1D)           (None, 250, 16)           64        
_________________________________________________________________
max_pooling1d_84 (MaxPooling (None, 125, 16)           0         
_________________________________________________________________
conv1d_85 (Conv1D)           (None, 123, 32)           1568      
_________________________________________________________________
max_pooling1d_85 (MaxPooling (None, 61, 32)            0         
____

### CNN With Normalized Input

In [40]:
(cnn2_best_score, cnn2_best_params, cnn2_best_estimator) = \
train_model(X_train, y_train, 'cnn_2', ncols, set_shape_create_cnn_model,
            pipeline_estimator_np, param_dist)

Model: "cnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_87 (Conv1D)           (None, 250, 16)           64        
_________________________________________________________________
max_pooling1d_87 (MaxPooling (None, 125, 16)           0         
_________________________________________________________________
conv1d_88 (Conv1D)           (None, 123, 32)           1568      
_________________________________________________________________
max_pooling1d_88 (MaxPooling (None, 61, 32)            0         
_________________________________________________________________
conv1d_89 (Conv1D)           (None, 59, 48)            4656      
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 29, 48)            0         
_________________________________________________________________
flatten_94 (Flatten)         (None, 1392)              0     

Epoch 5/100
45/45 [==============================] - 5s 122ms/step - loss: 14.3228 - accuracy: 0.3289 - val_loss: 9.5007 - val_accuracy: 0.3708
Epoch 6/100
45/45 [==============================] - 7s 151ms/step - loss: 6.8864 - accuracy: 0.2994 - val_loss: 3.7759 - val_accuracy: 0.3708
Epoch 7/100
45/45 [==============================] - 8s 169ms/step - loss: 9.6566 - accuracy: 0.3219 - val_loss: 15.0894 - val_accuracy: 0.3062
Epoch 8/100
45/45 [==============================] - 8s 169ms/step - loss: 9.9219 - accuracy: 0.3359 - val_loss: 16.4733 - val_accuracy: 0.3062
Epoch 9/100
45/45 [==============================] - 6s 142ms/step - loss: 10.8434 - accuracy: 0.3197 - val_loss: 13.2524 - val_accuracy: 0.3062
Epoch 10/100
45/45 [==============================] - 6s 138ms/step - loss: 10.4665 - accuracy: 0.3275 - val_loss: 16.8720 - val_accuracy: 0.3708
Epoch 11/100
45/45 [==============================] - 7s 152ms/step - loss: 10.5528 - accuracy: 0.3254 - val_loss: 1.6409 - val_accura

Epoch 1/100
45/45 [==============================] - 6s 141ms/step - loss: 11416369.0000 - accuracy: 0.3483 - val_loss: 14.6132 - val_accuracy: 0.2781
Epoch 2/100
45/45 [==============================] - 5s 106ms/step - loss: 9486.5107 - accuracy: 0.3364 - val_loss: 7.4599 - val_accuracy: 0.2781
Epoch 3/100
45/45 [==============================] - 4s 100ms/step - loss: 207.1864 - accuracy: 0.3490 - val_loss: 4.6036 - val_accuracy: 0.2781
Epoch 4/100
45/45 [==============================] - 8s 179ms/step - loss: 5.7263 - accuracy: 0.3378 - val_loss: 1.3734 - val_accuracy: 0.3455
Epoch 5/100
45/45 [==============================] - 5s 117ms/step - loss: 4.2712 - accuracy: 0.3413 - val_loss: 1.3316 - val_accuracy: 0.2781
Epoch 6/100
45/45 [==============================] - 6s 137ms/step - loss: 4.2917 - accuracy: 0.3244 - val_loss: 8.8263 - val_accuracy: 0.2781
Epoch 7/100
45/45 [==============================] - 6s 140ms/step - loss: 1177780.3750 - accuracy: 0.3350 - val_loss: 58.3512 - 

45/45 [==============================] - 1s 22ms/step - loss: 5.6689 - accuracy: 0.3254 - val_loss: 1.1935 - val_accuracy: 0.3062
Epoch 19/100
45/45 [==============================] - 1s 27ms/step - loss: 5.7550 - accuracy: 0.3176 - val_loss: 1.1808 - val_accuracy: 0.3062
Epoch 20/100
45/45 [==============================] - 1s 22ms/step - loss: 6.2164 - accuracy: 0.3289 - val_loss: 1.4225 - val_accuracy: 0.3062
Epoch 21/100
45/45 [==============================] - 1s 21ms/step - loss: 5.5946 - accuracy: 0.3099 - val_loss: 1.1824 - val_accuracy: 0.3062
Epoch 22/100
28/28 [==============================] - 0s 10ms/step - loss: 1.1248 - accuracy: 0.3157
Model: "cnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_109 (Conv1D)          (None, 250, 16)           64        
_________________________________________________________________
max_pooling1d_109 (MaxPoolin (None, 125, 16)           0     

Epoch 1/100
45/45 [==============================] - 4s 89ms/step - loss: 9723095.0000 - accuracy: 0.3504 - val_loss: 2259.7942 - val_accuracy: 0.3455
Epoch 2/100
45/45 [==============================] - 2s 35ms/step - loss: 2139.1804 - accuracy: 0.3322 - val_loss: 41.9204 - val_accuracy: 0.3455
Epoch 3/100
45/45 [==============================] - 1s 27ms/step - loss: 36.8651 - accuracy: 0.3546 - val_loss: 32.2995 - val_accuracy: 0.2781
Epoch 4/100
45/45 [==============================] - 1s 30ms/step - loss: 29.4488 - accuracy: 0.3315 - val_loss: 24.5000 - val_accuracy: 0.3455
Epoch 5/100
45/45 [==============================] - 1s 29ms/step - loss: 23.2470 - accuracy: 0.3455 - val_loss: 16.8937 - val_accuracy: 0.2781
Epoch 6/100
45/45 [==============================] - 1s 31ms/step - loss: 15.5084 - accuracy: 0.3406 - val_loss: 8.9536 - val_accuracy: 0.3455
Epoch 7/100
45/45 [==============================] - 1s 28ms/step - loss: 8.8048 - accuracy: 0.3237 - val_loss: 1.3447 - val_acc

...

Epoch 1/100
45/45 [==============================] - 8s 188ms/step - loss: 1.0871 - accuracy: 0.4024 - val_loss: 1.0413 - val_accuracy: 0.4185
Epoch 2/100
45/45 [==============================] - 7s 145ms/step - loss: 0.9676 - accuracy: 0.5499 - val_loss: 0.9530 - val_accuracy: 0.5225
Epoch 3/100
45/45 [==============================] - 7s 156ms/step - loss: 0.8781 - accuracy: 0.5829 - val_loss: 0.8512 - val_accuracy: 0.5927
Epoch 4/100
45/45 [==============================] - 6s 131ms/step - loss: 0.8351 - accuracy: 0.6152 - val_loss: 0.8547 - val_accuracy: 0.5702
Epoch 5/100
45/45 [==============================] - 5s 104ms/step - loss: 0.7939 - accuracy: 0.6566 - val_loss: 1.2550 - val_accuracy: 0.4242
Epoch 6/100
45/45 [==============================] - 4s 96ms/step - loss: 0.7927 - accuracy: 0.6496 - val_loss: 0.8331 - val_accuracy: 0.5899
Epoch 7/100
45/45 [==============================] - 4s 98ms/step - loss: 0.7650 - accuracy: 0.6573 - val_loss: 0.8000 - val_accuracy: 0.6152
E

45/45 [==============================] - 2s 39ms/step - loss: 0.9711 - accuracy: 0.5594 - val_loss: 0.9820 - val_accuracy: 0.5421
Epoch 75/100
45/45 [==============================] - 2s 41ms/step - loss: 0.9730 - accuracy: 0.5608 - val_loss: 0.9675 - val_accuracy: 0.5730
Epoch 76/100
45/45 [==============================] - 1s 31ms/step - loss: 0.9682 - accuracy: 0.5573 - val_loss: 0.9692 - val_accuracy: 0.5534
Epoch 77/100
45/45 [==============================] - 1s 32ms/step - loss: 0.9679 - accuracy: 0.5643 - val_loss: 0.9608 - val_accuracy: 0.5590
Epoch 78/100
45/45 [==============================] - 1s 29ms/step - loss: 0.9684 - accuracy: 0.5573 - val_loss: 0.9639 - val_accuracy: 0.5506
Epoch 79/100
45/45 [==============================] - 1s 31ms/step - loss: 0.9617 - accuracy: 0.5509 - val_loss: 0.9763 - val_accuracy: 0.5421
Epoch 80/100
45/45 [==============================] - 1s 29ms/step - loss: 0.9643 - accuracy: 0.5531 - val_loss: 0.9542 - val_accuracy: 0.5787
Epoch 81/100

45/45 [==============================] - 2s 38ms/step - loss: 0.9881 - accuracy: 0.5805 - val_loss: 0.9866 - val_accuracy: 0.5618
Epoch 75/100
45/45 [==============================] - 2s 35ms/step - loss: 0.9862 - accuracy: 0.5777 - val_loss: 0.9829 - val_accuracy: 0.5646
Epoch 76/100
45/45 [==============================] - 2s 39ms/step - loss: 0.9844 - accuracy: 0.5748 - val_loss: 0.9854 - val_accuracy: 0.5730
Epoch 77/100
45/45 [==============================] - 2s 38ms/step - loss: 0.9850 - accuracy: 0.5699 - val_loss: 0.9772 - val_accuracy: 0.5758
Epoch 78/100
45/45 [==============================] - 2s 36ms/step - loss: 0.9787 - accuracy: 0.5882 - val_loss: 0.9758 - val_accuracy: 0.5674
Epoch 79/100
45/45 [==============================] - 2s 34ms/step - loss: 0.9780 - accuracy: 0.5819 - val_loss: 0.9814 - val_accuracy: 0.5562
Epoch 80/100
45/45 [==============================] - 1s 32ms/step - loss: 0.9813 - accuracy: 0.5692 - val_loss: 0.9708 - val_accuracy: 0.5899
Epoch 81/100

45/45 [==============================] - 2s 36ms/step - loss: 0.9869 - accuracy: 0.5499 - val_loss: 1.0044 - val_accuracy: 0.5281
Epoch 75/100
45/45 [==============================] - 2s 38ms/step - loss: 0.9854 - accuracy: 0.5527 - val_loss: 1.0137 - val_accuracy: 0.5056
Epoch 76/100
45/45 [==============================] - 2s 35ms/step - loss: 0.9850 - accuracy: 0.5674 - val_loss: 1.0054 - val_accuracy: 0.5253
Epoch 77/100
45/45 [==============================] - 2s 40ms/step - loss: 0.9822 - accuracy: 0.5590 - val_loss: 1.0022 - val_accuracy: 0.5140
Epoch 78/100
45/45 [==============================] - 2s 39ms/step - loss: 0.9774 - accuracy: 0.5625 - val_loss: 0.9990 - val_accuracy: 0.5253
Epoch 79/100
45/45 [==============================] - 2s 38ms/step - loss: 0.9779 - accuracy: 0.5702 - val_loss: 1.0011 - val_accuracy: 0.5028
Epoch 80/100
45/45 [==============================] - 2s 40ms/step - loss: 0.9766 - accuracy: 0.5667 - val_loss: 0.9945 - val_accuracy: 0.5169
Epoch 81/100

Epoch 21/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7299 - accuracy: 0.6820 - val_loss: 0.7404 - val_accuracy: 0.6648
Epoch 22/100
67/67 [==============================] - 2s 31ms/step - loss: 0.7264 - accuracy: 0.6801 - val_loss: 0.7554 - val_accuracy: 0.6536
Epoch 23/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7166 - accuracy: 0.7012 - val_loss: 0.7664 - val_accuracy: 0.6479
Epoch 24/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7254 - accuracy: 0.6852 - val_loss: 0.7538 - val_accuracy: 0.6517
Epoch 25/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7156 - accuracy: 0.6904 - val_loss: 0.7358 - val_accuracy: 0.6667
Epoch 26/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7150 - accuracy: 0.6867 - val_loss: 0.7382 - val_accuracy: 0.6742
Epoch 27/100
67/67 [==============================] - 2s 30ms/step - loss: 0.7205 - accuracy: 0.7016 - val_loss: 0.7395 - val_accuracy: 0.6667